In [2]:
import pandas as pd
import os
import glob
import numpy as np
# import enchant
from collections import Counter
from matplotlib import pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder
# from tqdm import tqdm
%matplotlib inline
from collections import defaultdict
import math
import re
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
# import graphviz
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import KFold,StratifiedKFold
# from tqdm import tqdm
from sklearn import metrics
import enchant
np.random.seed(512)
# import subprocess
# subprocess.call(["pip","install", "pyenchant","--","user"])
# os.system('pip install pyenchant')
# import nltk
# from nltk.corpus import words
# nltk.download()

In [3]:
#read csv
dict_label = {'Usable directly numeric':0, 'Usable with extraction':1, 'Usable with Extration': 1, 'Usable with extraction ':1, 'Usable directly categorical':2, 'Unusable':3, 'Context_specific':4, 'Usable directly categorical ':2}
data = pd.read_csv('data_for_ML_num.csv')

data['y_act'] = [dict_label[i] for i in data['y_act']]
y = data.loc[:,['y_act']]

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (2,5,10,11,12,13,14,15,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
data['Num of nans'] = [data['Num of nans'][i]*100/data['Total_val'][i] for i in data.index]
data['num of dist_val'] = [data['num of dist_val'][i]*100/data['Total_val'][i] for i in data.index]

data1 = data[['Num of nans', 'max_val', 'mean', 'min_val', 'num of dist_val','std_dev','castability','extractability', 'len_val']]
data1 = data1.fillna(0)

# data = data.rename(columns={'mean': 'scaled_mean', 'min_val': 'scaled_min_val', 'max_val': 'scaled_max_val','std_dev': 'scaled_std_dev','len_val': 'scaled_len_val'})
# column_names_to_normalize = ['scaled_max_val', 'scaled_mean', 'scaled_min_val','scaled_std_dev', 'scaled_len_val']
# x = data[column_names_to_normalize].values
# x = np.nan_to_num(x)
# x_scaled = StandardScaler().fit_transform(x)
# df_temp = pd.DataFrame(x_scaled, columns=column_names_to_normalize, index = data.index)
# data[column_names_to_normalize] = df_temp

# X_train, X_test,y_train,y_test = train_test_split(data,y, test_size=0.2)

# new_data = data[['scaled_min_val']]
# new_data.scaled_min_val = new_data.scaled_min_val.astype(float)
# normalized_df=(new_data-new_data.mean())/new_data.std()
# new_data = new_data[np.abs(new_data.scaled_min_val-new_data.scaled_min_val.mean()) <= (3*new_data.scaled_min_val.std())]

# for index, row in new_data.iterrows():
#     if row['scaled_min_val'] > 100000:
#         new_data.iloc[index]['scaled_min_val'] = 100000
#     if row['scaled_min_val'] < -100000:
#         new_data.iloc[index]['scaled_min_val'] = -100000        
#     print(row['scaled_min_val']) 

# q = new_data['scaled_min_val'].quantile(0.99)
# new_data = new_data[new_data['scaled_min_val'] < q]
# print(new_data)

# print(new_data.mean())
# print(new_data.std())


# scaled_features = StandardScaler().fit_transform(new_data.values)
# df = pd.DataFrame(scaled_features)
# scaled_features = MinMaxScaler().fit_transform(new_data.values)
# df = pd.DataFrame(scaled_features)

# std_scale = preprocessing.StandardScaler().fit(data[['max_val', 'min_val']])
# data = std_scale.transform(data[['max_val', 'min_val']])
# data = pd.DataFrame(data)

In [5]:
# data1.to_csv('before.csv')
# f = open('current.txt','w')
d = enchant.Dict("en_US")

for i in data.index:
    ival = data.at[i,'Attribute_name']
    if ival != 'id' and d.check(ival):
#         print >> f,ival
#         print >> f,y.at[i,'y_act']
        data1.at[i,'dictionary_item'] = 1
    else:
        data1.at[i,'dictionary_item'] = 0

# data1.to_csv('after.csv')
# f.close()

In [6]:
# print(data1.columns)

arr = data['Attribute_name'].values

vectorizer = CountVectorizer(ngram_range=(2,2),analyzer='char')
X = vectorizer.fit_transform(arr)
print(len(vectorizer.get_feature_names()))
# print(X.toarray())

# data1.to_csv('before.csv')

tempdf = pd.DataFrame(X.toarray())

data2 = pd.concat([data1,tempdf], axis=1, sort=False)

# data2.to_csv('after.csv')

1286


In [10]:
X_train, X_test,y_train,y_test = train_test_split(data2,y, test_size=0.2,random_state=100)
print(len(X_train))
data2 = data2.rename(columns={'mean': 'scaled_mean', 'min_val': 'scaled_min_val', 'max_val': 'scaled_max_val','std_dev': 'scaled_std_dev'})


X_train_new = X_train.reset_index(drop=True)
y_train_new = y_train.reset_index(drop=True)
# print(X_train.head())
# print(y_train.head())

X_train_new = X_train_new.values
y_train_new = y_train_new.values
# print(X_train_new)
# print(y_train_new)

k = 5
kf = KFold(n_splits=k)
avg_train_acc,avg_test_acc = 0,0
    
n_estimators_grid = [5,25,50,75,100]
max_depth_grid = [5,10,25,50,100]

avgsc_lst,avgsc_train_lst,avgsc_hld_lst = [],[],[]
avgsc,avgsc_train,avgsc_hld = 0,0,0

for train_index, test_index in kf.split(X_train_new):
    X_train_cur, X_test_cur = X_train_new[train_index], X_train_new[test_index]
    y_train_cur, y_test_cur = y_train_new[train_index], y_train_new[test_index]
    X_train_train, X_val,y_train_train,y_val = train_test_split(X_train_cur,y_train_cur, test_size=0.25,random_state=100)
    
    bestPerformingModel = RandomForestClassifier(n_estimators=10,max_depth=5)
    bestscore = 0
    for ne in n_estimators_grid:
        for md in max_depth_grid:
            clf = RandomForestClassifier(n_estimators=ne,max_depth=md)
            clf.fit(X_train_train, y_train_train)
            sc = clf.score(X_val, y_val)
    
            if bestscore < sc:
                bestscore = sc
                bestPerformingModel = clf
#                 print(bestPerformingModel)

    bscr_train = bestPerformingModel.score(X_train_cur, y_train_cur)
    bscr = bestPerformingModel.score(X_test_cur, y_test_cur)
    bscr_hld = bestPerformingModel.score(X_test, y_test)

    avgsc_train_lst.append(bscr_train)
    avgsc_lst.append(bscr)
    avgsc_hld_lst.append(bscr_hld)
    
    avgsc_train = avgsc_train + bscr_train    
    avgsc = avgsc + bscr
    avgsc_hld = avgsc_hld + bscr_hld

    print(bscr_train)
    print(bscr)
    print(bscr_hld)

7400


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:36: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


0.9677364864864865
0.8912162162162162
0.8978930307941653
0.9675675675675676
0.8695945945945946
0.8833063209076175
0.9721283783783784
0.8858108108108108
0.8914100486223663
0.972804054054054
0.8945945945945946
0.8941112911939492
0.9716216216216216
0.8831081081081081
0.8946515397082658


In [11]:
print(avgsc_train_lst)
print(avgsc_lst)
print(avgsc_hld_lst)

print(avgsc_train/k)
print(avgsc/k)
print(avgsc_hld/k)

[0.9677364864864865, 0.9675675675675676, 0.9721283783783784, 0.972804054054054, 0.9716216216216216]
[0.8912162162162162, 0.8695945945945946, 0.8858108108108108, 0.8945945945945946, 0.8831081081081081]
[0.8978930307941653, 0.8833063209076175, 0.8914100486223663, 0.8941112911939492, 0.8946515397082658]
0.9703716216216216
0.8848648648648648
0.8922744462452729
